In [1]:
import static_graph_construction

OSError: [WinError 127] The specified procedure could not be found

In [9]:
import torch

torch.load('../evaluation/test-1790-qs.pt')

ModuleNotFoundError: No module named 'static_graph_construction'